... imports

In [1]:
import pandas as pd
import numpy as np
import os
import spacy
from spacy.lang.de import German
from spacy.lang.en import English
from spacy import displacy

from spacy.lang.de.stop_words import STOP_WORDS
nlp = spacy.load('de_core_news_sm')

***
## Text Preprocessing mit Spacy

### Online-Artikel aus auto.motorsport einlesen und tokenizen

In [2]:
txt = open("Vierer.txt", encoding="utf-8").read()
doc = nlp(txt); doc

for i,sent in enumerate(doc.sents):
    print(i,sent)
    
# print(txt)

0 BMW Concept 4
1 Series Coupé


2 Auf der IAA zeigt BMW das Concept 4 Series Coupé.
3 Es gibt einen Ausblick auf den neuen 4er, der in der zweiten Generation 2020 eine komplett neue Front bekommt und noch deutlich sportlicher wird.

4 Die große vertikale Niere ist zurück.
5 BMW zeigt sie erstmals auf der Internationalen Automobilausstelung (IAA) in Frankfurt.
6 Am BMW Concept 4
7 Series Coupé ist sie nicht mehr – wie bei den aktuellen BMW Modellen gängig – breit und flach gehalten, sondern reicht von der Motorhaube bis über die untere Frontschürze.
8 Laut BMW ist das Konzeptfahrzeug „eine Vorkommunikation für das, was in der Serie kommen könnte“.
9 Die Bayern bleiben bewusst im Konjunktiv, fügen aber hinzu:
10 „Das Concept gibt einen ca. 80-90-Prozent-Ausblick auf die Serie.“


11 Hinsichtlich der Front schafft BMW offiziell Klarheit darüber, dass hier das neue 4er-Gesicht zu sehen ist.
12 Wir erwarten es schon beim 2020er Coupé.
13 Es soll, so der Münchner Hersteller, „dem Fahrzeug e

Beim Einlesen dem sog. "Parsen" wird ein vortrainiertes NLP-Modell auf die Daten angewandt.  
Dadurch werden die einzelnen Elemente des Textes automatisch bereits klassifiziert:  

In [3]:
bspDisp = nlp("Die große vertikale Niere ist zurück.")
displacy.render(bspDisp, style=("dep"))

Natürlich liegen diese Informationen für jedes "Token" (hier jedes Wort) vor:

In [4]:
bmwToken = pd.DataFrame({"TOKEN":[], "LEMMA":[], "POS":[], "TAG":[], "DEP":[], "SHAPE":[], "ALPHA":[],  "STOP":[],})
for i,token in enumerate(doc):
    tokenFeatures = [token.text,	token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop,]
    bmwToken.loc[i,:] = tokenFeatures

bmwToken.drop_duplicates(subset="TOKEN", inplace=True)
print(len(bmwToken))
bmwToken.sample(20)

480


,TOKEN,LEMMA,POS,TAG,DEP,SHAPE,ALPHA,STOP
535,indes,indes,ADV,ADV,mo,xxxx,True,False
8,IAA,IAA,PROPN,NE,nk,XXX,True,False
353,einer,einer,DET,ART,nk,xxxx,True,True
578,Weg,Weg,NOUN,NN,oa,Xxx,True,False
242,sportlichste,sportlich,ADJ,ADJA,nk,xxxx,True,False
731,Ihre,mein,DET,PPOSAT,nk,Xxxx,True,True
320,natürlich,natürlich,ADV,ADV,mo,xxxx,True,True
293,Vieraugen-Scheinwerfern,Vieraugen-Scheinwerfern,NOUN,NN,nk,Xxxxx-Xxxxx,False,False
609,Schluss,Schluss,PROPN,NE,sb,Xxxxx,True,False
396,Vorbild,Vorbild,NOUN,NN,nk,Xxxxx,True,False


In [5]:
bmwToken.sample(20, random_state=111)

,TOKEN,LEMMA,POS,TAG,DEP,SHAPE,ALPHA,STOP
456,Dahinter,Dahinter,ADV,PROAV,mo,Xxxxx,True,True
823,Schwarz,Schwarz,NOUN,NN,nk,Xxxxx,True,False
901,PS,PS,NOUN,NN,nk,XX,True,False
483,Sie,ich,PRON,PPER,sb,Xxx,True,True
742,brachiale,brachiale,ADJ,ADJA,nk,xxxx,True,False
166,schon,schon,ADV,ADV,mo,xxxx,True,True
184,Charakter,Charakter,NOUN,NN,oa,Xxxxx,True,False
21,auf,auf,ADP,APPR,mnr,xxx,True,True
336,Konzeptfahrzeugs,Konzeptfahrzeugs,NOUN,NN,ag,Xxxxx,True,False
194,sind,sein,AUX,VAFIN,ROOT,xxxx,True,True


### Stoppwörter rausfiltern

Stoppwort Liste erstellen

In [6]:
stopwordsDE = list(STOP_WORDS)
print("Anzahl: ",len(stopwordsDE)); print(stopwordsDE[:50])

Anzahl:  545
['können', 'fünfter', 'ein', 'manches', 'seid', 'ja', 'auf', 'kurz', 'danach', 'seine', 'grossen', 'musste', 'zweiten', 'muß', 'demzufolge', 'solchem', 'wurden', 'er', 'hin', 'wenn', 'eigen', 'viele', 'sollen', 'warum', 'sechstes', 'welchen', 'dieses', 'nachdem', 'trotzdem', 'teil', 'bist', 'offen', 'dies', 'jedermanns', 'keiner', 'magst', 'wurde', 'nichts', 'einem', 'deshalb', 'damals', 'aller', 'neunten', 'sonst', 'allein', 'drin', 'siebenten', 'hier', 'drittes', 'mögt']


In [7]:
bsp = nlp("Dieser Satz strotzt nur so vor lauter Stoppwörtern und es macht nicht nur Spaß sie alle zu finden ")
for token in bsp:
    if token.is_stop==True:
        print(token.text, token.pos_, token.is_stop)

Dieser DET True
nur ADV True
so ADV True
vor ADP True
und CCONJ True
es PRON True
macht VERB True
nicht PART True
nur ADV True
sie PRON True
alle PRON True
zu PART True


In [8]:
NoStopWordsDoc = [token for token in doc if token.is_stop == False]
print(NoStopWordsDoc[:100])

[BMW, Concept, 4, Series, Coupé, 

, IAA, zeigt, BMW, Concept, 4, Series, Coupé, ., Ausblick, 4er, ,, Generation, 2020, komplett, Front, bekommt, deutlich, sportlicher, ., 
, vertikale, Niere, ., BMW, zeigt, erstmals, Internationalen, Automobilausstelung, (, IAA, ), Frankfurt, ., BMW, Concept, 4, Series, Coupé, –, aktuellen, BMW, Modellen, gängig, –, breit, flach, gehalten, ,, reicht, Motorhaube, untere, Frontschürze, ., Laut, BMW, Konzeptfahrzeug, „, Vorkommunikation, ,, Serie, “, ., Bayern, bleiben, bewusst, Konjunktiv, ,, fügen, hinzu, :, „, Concept, ca., 80, -, 90-Prozent-Ausblick, Serie, ., “, 

, Hinsichtlich, Front, schafft, BMW, offiziell, Klarheit, ,, 4er-Gesicht, sehen, ., erwarten, 2020er, Coupé, .]


### Lemmatisieren

In [9]:
bspLemma = nlp('sehen gesehen sah sieht saht sahen seht siehste')
for lem in bspLemma: print(lem.text, "LEMMA ==> ", lem.lemma_)

sehen LEMMA ==>  sehen
gesehen LEMMA ==>  sehen
sah LEMMA ==>  sehen
sieht LEMMA ==>  sehen
saht LEMMA ==>  sehen
sahen LEMMA ==>  sehen
seht LEMMA ==>  sehen
siehste LEMMA ==>  siehste


### Entity Detection

In [10]:
entities = pd.DataFrame({"text":[], "start_char":[],"end_char":[],"label":[]})
for i, ent in enumerate(doc.ents):
    vars = [ent.text, ent.start_char, ent.end_char, ent.label_]
    # print(vars)
    entities.loc[i,:] = vars
    
entities.drop_duplicates(subset="text", inplace=True)
print(len(entities))
print(entities.label.unique())
print(entities.text.unique())

44
['MISC' 'ORG' 'LOC' 'PER']
['BMW Concept 4' 'IAA' 'BMW' 'Concept 4 Series Coupé'
 'Internationalen Automobilausstelung' 'Frankfurt' 'BMW Modellen' '“'
 'Bayern' 'Münchner' 'Concept' 'Concept 4' 'CLAR-Plattform' 'Außenmaße'
 'Vieraugen-Scheinwerfern' 'LED-Lichtanlage' 'BMW 328' 'E21'
 'Erlkönig-Bilder' 'Nordschleife' 'BMW-Designer' 'Serien-4er'
 'Erlkönig-Bildern' 'Spiegel' 'Straße' 'Schicke'
 'Luftauslässen hinter der Vorderachse' 'anatomisch' 'thront' 'L-förmigen'
 'Scheinwerfer' 'Heartbeat-Optik“' 'Red“' 'Tiefrot' 'Betrachtungswinkel'
 'BMW 4er-Studie' '3er-Regal' 'Benziner' 'Diesel' 'Hybrid' 'Topmodell M4'
 'Twin-Turbo' 'Competition-Version' 'AMG']


In [11]:
doc = nlp("New York City on Tuesday declared a public health emergency and ordered mandatory measles vaccinations amid an outbreak, becoming the latest national flash point over refusals to inoculate against dangerous diseases. At least 285 people have contracted measles in the city since September, mostly in Brooklyn’s Williamsburg neighborhood. The order covers four Zip codes there, Mayor Bill de Blasio (D) said Tuesday. The mandate orders all unvaccinated people in the area, including a concentration of Orthodox Jews, to receive inoculations, including for children as young as 6 months old. Anyone who resists could be fined up to $1,000.")
doc

New York City on Tuesday declared a public health emergency and ordered mandatory measles vaccinations amid an outbreak, becoming the latest national flash point over refusals to inoculate against dangerous diseases. At least 285 people have contracted measles in the city since September, mostly in Brooklyn’s Williamsburg neighborhood. The order covers four Zip codes there, Mayor Bill de Blasio (D) said Tuesday. The mandate orders all unvaccinated people in the area, including a concentration of Orthodox Jews, to receive inoculations, including for children as young as 6 months old. Anyone who resists could be fined up to $1,000.

In [12]:
displacy.render(doc, style="ent", page=True,)

***
## Text Classification auf Basis von YELP, amazon & IMDB Reviews

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix 

Daten laden

In [14]:
pd.set_option('display.max_colwidth', -1)
data = pd.read_csv("amazonYelpImdb.csv")
data.sample(10)

,Review,Sentiment
884,"On three different occasions I asked for well done or medium well, and all three times I got the bloodiest piece of meat on my plate.",0
1797,A good quality bargain.. I bought this after I bought a cheapy from Big Lots that sounded awful and people on the other end couldn't hear me.,1
1958,I really like this product over the Motorola because it is allot clearer on the ear piece and the mic.,1
2293,"This movie creates its own universe, and is fascinating in every way.",1
2394,"Being a 90's child, I truly enjoyed this show and I can proudly say that I enjoyed it big time and even more than the classical WB cartoons.",1
1071,"Even in my BMW 3 series which is fairly quiet, I have trouble hearing what the other person is saying.",0
2011,"The movie showed a lot of Florida at it's best, made it look very appealing.",1
2344,"While you don't yet hear Mickey speak, there are tons of sound effects and music throughout the film--something we take for granted now but which was a huge crowd pleaser in 1928.",1
1658,Talk about USELESS customer service.,0
640,The Macarons here are insanely good.,1


In [15]:
data.Sentiment.value_counts()

1    1386
0    1362
Name: Sentiment, dtype: int64

Preprocessing Function, die jeden einzelnen Review bearbeitet

In [16]:
from spacy.lang.en.stop_words import STOP_WORDS
import string
punct = string.punctuation
stopwords = list(STOP_WORDS)

def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens

In [17]:
text_data_cleaning("Martin, Herbert & Susanne are going to enjoy this afternoon at BMW after their heavy lunch break")

['martin',
 'herbert',
 'susanne',
 'going',
 'enjoy',
 'afternoon',
 'bmw',
 'heavy',
 'lunch',
 'break']

#### Vectorization Feature Engineering (TF-IDF)

In [18]:
from sklearn.svm import LinearSVC
tfidf = TfidfVectorizer(tokenizer=text_data_cleaning)
classifier = LinearSVC()

X,y = data["Review"], data["Sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test,y_pred, target_names=["Negativ", "Positiv"]))


              precision    recall  f1-score   support

     Negativ       0.79      0.83      0.81       285
     Positiv       0.81      0.77      0.79       265

    accuracy                           0.80       550
   macro avg       0.80      0.80      0.80       550
weighted avg       0.80      0.80      0.80       550



In [19]:
newText_1 = "Will not buy a Mercedes again"
newText_2 = "The new BMW is an absolutely wonderful experience"
clf.predict([newText_1, newText_2,])

array([0, 1], dtype=int64)

# Parkplatz

In [20]:
url = "https://raw.githubusercontent.com/strategiepilot/NLP-Tutorial-8---Sentiment-Classification-using-SpaCy-for-IMDB-and-Amazon-Review-Dataset/master/datasets/"
columns_name = ['Review', 'Sentiment']
data_yelp = pd.read_csv(url+"yelp_labelled.txt", sep='\t', header = None, names=columns_name)            ; print(data_yelp.shape)
data_amazon = pd.read_csv(url+"amazon_cells_labelled.txt", sep='\t', header = None, names=columns_name)  ; print(data_amazon.shape)
data_imdb = pd.read_csv(url+"imdb_labelled.txt", sep='\t', header = None, names=columns_name)            ; print(data_imdb.shape)
data = pd.concat([data_yelp, data_amazon, data_imdb], axis=0, ignore_index=True, )                       ; print(data.shape)
data.head()

(1000, 2)
(1000, 2)
(748, 2)
(2748, 2)


,Review,Sentiment
0,Wow... Loved this place.,1
1,Crust is not good.,0
2,Not tasty and the texture was just nasty.,0
3,Stopped by during the late May bank holiday off Rick Steve recommendation and loved it.,1
4,The selection on the menu was great and so were the prices.,1
